# Project description

The goal was to make a model for predicting electricity demand, based on ENTSO-E electricity load data. The data is available at the link below.

https://eepublicdownloads.blob.core.windows.net/public-cdn-container/clean-documents/Publications/Statistics/MHLV_data-2015-2017.xlsx

## Data download

In [1]:
import pandas as pd
import nltk
import re
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OrdinalEncoder 
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.neighbors import KNeighborsRegressor

In [2]:
%%time
df = pd.read_excel(r'C:\Users\Рус\Desktop\python\Новая папка\MHLV_data-2015-2017.xlsx')

print(df.keys())

print(df.info())
print('\n')

print(df)


Index(['MeasureItem', 'DateUTC', 'DateShort', 'TimeFrom', 'TimeTo',
       'CountryCode', 'Cov_ratio', 'Value', 'Value_ScaleTo100'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631600 entries, 0 to 631599
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   MeasureItem       631600 non-null  object        
 1   DateUTC           631600 non-null  datetime64[ns]
 2   DateShort         631600 non-null  datetime64[ns]
 3   TimeFrom          631600 non-null  object        
 4   TimeTo            631600 non-null  object        
 5   CountryCode       631600 non-null  object        
 6   Cov_ratio         631600 non-null  int64         
 7   Value             631600 non-null  float64       
 8   Value_ScaleTo100  631600 non-null  float64       
dtypes: datetime64[ns](2), float64(2), int64(1), object(4)
memory usage: 43.4+ MB
None


                       MeasureItem       

## Preprocessing

In [3]:
df['CountryCode'].unique()

array(['DE', 'BG', 'EE', 'FI', 'GR', 'LT', 'LV', 'RO', 'TR', 'AT', 'BA',
       'BE', 'CH', 'CY', 'CZ', 'DK', 'ES', 'FR', 'HR', 'HU', 'IT', 'LU',
       'ME', 'MK', 'NL', 'NO', 'PL', 'RS', 'SE', 'SI', 'SK', 'GB', 'IE',
       'IS', 'PT', 'AL'], dtype=object)

Selecting table data by Country Code column. I need only one country data. So i chose DE or Denmark.

In [4]:
df_de = df.query('CountryCode == "DE"')

Removing all extra columns.

In [5]:
df_de = df_de.drop(['MeasureItem','DateShort','TimeFrom','TimeTo','CountryCode','Cov_ratio','Value_ScaleTo100'], axis = 1)
print(df_de.head(5))

              DateUTC     Value
0 2014-12-31 23:00:00  46419.79
1 2015-01-01 00:00:00  44898.30
2 2015-01-01 01:00:00  43305.31
3 2015-01-01 02:00:00  41918.17
4 2015-01-01 03:00:00  41330.17


In [6]:
df_de.describe()

,Value
count,26305.000000
mean,57760.229774
std,10222.452224
min,35084.520000
25%,49097.050000
50%,57448.620000
75%,66711.240000
max,79487.370000


Turning the DateUTC column into index.

In [7]:
%%time
df_de['DateUTC'] = pd.to_datetime(df_de['DateUTC'], format='%Y-%m-%dT%H:%M:%S')
df_de = df_de.set_index('DateUTC')
print(df_de.index.is_monotonic)

True
Wall time: 10 ms


In [8]:
print(df_de.head(5))

                        Value
DateUTC                      
2014-12-31 23:00:00  46419.79
2015-01-01 00:00:00  44898.30
2015-01-01 01:00:00  43305.31
2015-01-01 02:00:00  41918.17
2015-01-01 03:00:00  41330.17


A function that creates features for machine learning.

In [9]:
%%time
def make_features(df, max_lag):
    df_de['year'] = df_de.index.year
    df_de['month'] = df_de.index.month
    df_de['day'] = df_de.index.day
    df_de['dayofweek'] = df_de.index.dayofweek
    df_de['hournmb'] = df_de.index.hour
    
    for lag in range(1, max_lag + 1):
        df_de['lag_{}'.format(lag)] = df_de['Value'].shift(lag)
    

make_features(df, 23)

Wall time: 23.5 ms


In [10]:
df_de.head(5)

,Value,year,month,day,dayofweek,hournmb,lag_1,lag_2,lag_3,lag_4,...,lag_14,lag_15,lag_16,lag_17,lag_18,lag_19,lag_20,lag_21,lag_22,lag_23
DateUTC,,,,,,,,,,,,,,,,,,,,,
2014-12-31 23:00:00,46419.79,2014,12,31,2,23,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 00:00:00,44898.30,2015,1,1,3,0,46419.79,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 01:00:00,43305.31,2015,1,1,3,1,44898.30,46419.79,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 02:00:00,41918.17,2015,1,1,3,2,43305.31,44898.30,46419.79,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 03:00:00,41330.17,2015,1,1,3,3,41918.17,43305.31,44898.30,46419.79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_de = df_de.dropna()

In [12]:
df_de

,Value,year,month,day,dayofweek,hournmb,lag_1,lag_2,lag_3,lag_4,...,lag_14,lag_15,lag_16,lag_17,lag_18,lag_19,lag_20,lag_21,lag_22,lag_23
DateUTC,,,,,,,,,,,,,,,,,,,,,
2015-01-01 22:00:00,47987.90,2015,1,1,3,22,50802.46,51026.32,52220.90,54365.26,...,42361.15,40049.74,39372.50,39132.17,40643.13,41330.17,41918.17,43305.31,44898.30,46419.79
2015-01-01 23:00:00,45267.68,2015,1,1,3,23,47987.90,50802.46,51026.32,52220.90,...,44998.15,42361.15,40049.74,39372.50,39132.17,40643.13,41330.17,41918.17,43305.31,44898.30
2015-01-02 00:00:00,43332.57,2015,1,2,4,0,45267.68,47987.90,50802.46,51026.32,...,48057.62,44998.15,42361.15,40049.74,39372.50,39132.17,40643.13,41330.17,41918.17,43305.31
2015-01-02 01:00:00,42373.20,2015,1,2,4,1,43332.57,45267.68,47987.90,50802.46,...,49133.93,48057.62,44998.15,42361.15,40049.74,39372.50,39132.17,40643.13,41330.17,41918.17
2015-01-02 02:00:00,42208.23,2015,1,2,4,2,42373.20,43332.57,45267.68,47987.90,...,48444.78,49133.93,48057.62,44998.15,42361.15,40049.74,39372.50,39132.17,40643.13,41330.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 19:00:00,49041.19,2017,12,31,6,19,52369.34,55217.78,55723.32,52701.17,...,37803.14,37705.71,37789.72,38004.46,38869.25,40696.11,43386.63,47122.58,50689.38,52207.48
2017-12-31 20:00:00,47306.32,2017,12,31,6,20,49041.19,52369.34,55217.78,55723.32,...,39599.65,37803.14,37705.71,37789.72,38004.46,38869.25,40696.11,43386.63,47122.58,50689.38
2017-12-31 21:00:00,47068.99,2017,12,31,6,21,47306.32,49041.19,52369.34,55217.78,...,42237.75,39599.65,37803.14,37705.71,37789.72,38004.46,38869.25,40696.11,43386.63,47122.58


In [13]:
df_de.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26282 entries, 2015-01-01 22:00:00 to 2017-12-31 23:00:00
Data columns (total 29 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Value      26282 non-null  float64
 1   year       26282 non-null  int64  
 2   month      26282 non-null  int64  
 3   day        26282 non-null  int64  
 4   dayofweek  26282 non-null  int64  
 5   hournmb    26282 non-null  int64  
 6   lag_1      26282 non-null  float64
 7   lag_2      26282 non-null  float64
 8   lag_3      26282 non-null  float64
 9   lag_4      26282 non-null  float64
 10  lag_5      26282 non-null  float64
 11  lag_6      26282 non-null  float64
 12  lag_7      26282 non-null  float64
 13  lag_8      26282 non-null  float64
 14  lag_9      26282 non-null  float64
 15  lag_10     26282 non-null  float64
 16  lag_11     26282 non-null  float64
 17  lag_12     26282 non-null  float64
 18  lag_13     26282 non-null  float64
 19  lag_14     

In [14]:
print(df_de.index.is_monotonic)

True


Dividing the table into two selections.

In [15]:
%%time
train, test = train_test_split(df_de, shuffle=False, test_size=0.2)

Wall time: 4 ms


In [16]:
%%time
target_train = train['Value']
features_train = train.drop('Value', axis=1)
target_test = test['Value']
features_test = test.drop('Value', axis=1)

Wall time: 3 ms


# Model training

In [17]:
%%time
model = LinearRegression()
model.fit(features_train, target_train)
pred = model.predict(features_test)
print("RMSE:", sqrt(mean_squared_error(target_test, pred)))

RMSE: 1033.1741740998373
Wall time: 271 ms


In [18]:
print('RMSD as a percentage of the average demand value')
print((sqrt(mean_squared_error(target_test, pred))/df_de['Value'].mean())*100)

RMSD as a percentage of the average demand value
1.7884290881103941


In [19]:
df_de['Value'].mean()

57769.92674568167

In [20]:
sqrt(mean_squared_error(target_test, pred))

1033.1741740998373

In [21]:
pred

array([41221.75892326, 40812.054516  , 39449.659762  , ...,
       45838.22458387, 45327.15599272, 41440.11362456])

In [22]:
target_test

DateUTC
2017-05-26 23:00:00    41693.08
2017-05-27 00:00:00    40375.44
2017-05-27 01:00:00    39966.05
2017-05-27 02:00:00    39900.54
2017-05-27 03:00:00    39988.21
                         ...   
2017-12-31 19:00:00    49041.19
2017-12-31 20:00:00    47306.32
2017-12-31 21:00:00    47068.99
2017-12-31 22:00:00    44983.93
2017-12-31 23:00:00    44407.63
Name: Value, Length: 5257, dtype: float64

In [23]:
%%time
tscv = TimeSeriesSplit()
clf = RandomForestRegressor()
parametrs = {'n_estimators': range (9, 19, 2), 'max_depth': range (9, 19, 2)}
grid = GridSearchCV(clf, parametrs, cv=tscv)
grid.fit(features_train, target_train)
grid.best_params_

Wall time: 3min 42s


{'max_depth': 17, 'n_estimators': 17}

In [24]:
%%time
model = RandomForestRegressor(**grid.best_params_, random_state=12345)
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print(sqrt(mean_squared_error(target_test, predictions)))

507.5354380876218
Wall time: 5.43 s


In [25]:
print('RMSD as a percentage of the average demand value')
print((sqrt(mean_squared_error(target_test, predictions))/df_de['Value'].mean())*100)

RMSD as a percentage of the average demand value
0.8785460994643902


In [26]:
df_de

,Value,year,month,day,dayofweek,hournmb,lag_1,lag_2,lag_3,lag_4,...,lag_14,lag_15,lag_16,lag_17,lag_18,lag_19,lag_20,lag_21,lag_22,lag_23
DateUTC,,,,,,,,,,,,,,,,,,,,,
2015-01-01 22:00:00,47987.90,2015,1,1,3,22,50802.46,51026.32,52220.90,54365.26,...,42361.15,40049.74,39372.50,39132.17,40643.13,41330.17,41918.17,43305.31,44898.30,46419.79
2015-01-01 23:00:00,45267.68,2015,1,1,3,23,47987.90,50802.46,51026.32,52220.90,...,44998.15,42361.15,40049.74,39372.50,39132.17,40643.13,41330.17,41918.17,43305.31,44898.30
2015-01-02 00:00:00,43332.57,2015,1,2,4,0,45267.68,47987.90,50802.46,51026.32,...,48057.62,44998.15,42361.15,40049.74,39372.50,39132.17,40643.13,41330.17,41918.17,43305.31
2015-01-02 01:00:00,42373.20,2015,1,2,4,1,43332.57,45267.68,47987.90,50802.46,...,49133.93,48057.62,44998.15,42361.15,40049.74,39372.50,39132.17,40643.13,41330.17,41918.17
2015-01-02 02:00:00,42208.23,2015,1,2,4,2,42373.20,43332.57,45267.68,47987.90,...,48444.78,49133.93,48057.62,44998.15,42361.15,40049.74,39372.50,39132.17,40643.13,41330.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 19:00:00,49041.19,2017,12,31,6,19,52369.34,55217.78,55723.32,52701.17,...,37803.14,37705.71,37789.72,38004.46,38869.25,40696.11,43386.63,47122.58,50689.38,52207.48
2017-12-31 20:00:00,47306.32,2017,12,31,6,20,49041.19,52369.34,55217.78,55723.32,...,39599.65,37803.14,37705.71,37789.72,38004.46,38869.25,40696.11,43386.63,47122.58,50689.38
2017-12-31 21:00:00,47068.99,2017,12,31,6,21,47306.32,49041.19,52369.34,55217.78,...,42237.75,39599.65,37803.14,37705.71,37789.72,38004.46,38869.25,40696.11,43386.63,47122.58


In [27]:
%%time
tscv = TimeSeriesSplit()
clf = LGBMRegressor()
parametrs = {'n_estimators': range (9, 19, 2), 'max_depth': range (15, 25, 2)}
grid = GridSearchCV(clf, parametrs, cv=tscv)
grid.fit(features_train, target_train)
grid.best_params_

Wall time: 7.36 s


{'max_depth': 15, 'n_estimators': 17}

In [28]:
%%time
model = LGBMRegressor(**grid.best_params_, random_state=12345)
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print(sqrt(mean_squared_error(target_test, predictions)))

2039.9348362958397
Wall time: 105 ms


In [29]:
print('RMSD as a percentage of the average demand value')
print((sqrt(mean_squared_error(target_test, predictions))/df_de['Value'].mean())*100)

RMSD as a percentage of the average demand value
3.5311362696999193


In [30]:
%%time
tscv = TimeSeriesSplit()
clf = KNeighborsRegressor()
parametrs = {'n_neighbors': range (1, 10, 2), 'leaf_size': range (1, 10, 2)}
grid = GridSearchCV(clf, parametrs, cv=tscv)
grid.fit(features_train, target_train)
grid.best_params_

Wall time: 1min


{'leaf_size': 1, 'n_neighbors': 3}

In [31]:
%%time
model = KNeighborsRegressor(**grid.best_params_)
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print(sqrt(mean_squared_error(target_test, predictions)))

863.2220750847735
Wall time: 1.37 s


In [32]:
print('RMSD as a percentage of the average demand value')
print((sqrt(mean_squared_error(target_test, predictions))/df_de['Value'].mean())*100)

RMSD as a percentage of the average demand value
1.4942412492314605


# Сonclusions

The best result (RMSD 507.53 or 0.878 as a percentage of the average demand value) was achieved through the use of the RandomForestRegressor algorithm. Better results may be achived by examing wider grid parameters range< using different algoritms and features scaling.